In [10]:
# Now, install the dependencies
!pip install datasets
!pip install torch


In [11]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from huggingface_hub import notebook_login
from tqdm.auto import tqdm

# --- Step 1: Login to Hugging Face ---
# A login box will appear. Paste your Hugging Face access token here.
# Get a token at: https://huggingface.co/settings/tokens
print("Please log in to your Hugging Face account.")
notebook_login()


Please log in to your Hugging Face account.


In [6]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm

# --- Step 2: Configuration ---
PROJECT_PATH = ""
OUTPUT_DIR = os.path.join(PROJECT_PATH, "data")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ✅ Collect ~1 million tokens per language
TARGET_TOKEN_COUNT = 1_000_000
MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']
CONFIG_DATE = '20231101'

print(f"Output directory for text files: {OUTPUT_DIR}")

# --- Step 3: Load the Tokenizer ---
print(f"Loading tokenizer for '{MODEL_NAME}'...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✅ Tokenizer loaded successfully.")

# --- Step 4: Process each language ---
for lang in LANGUAGES:
    print("\n" + "=" * 60)
    print(f"🌍 Processing language: {lang}")
    print("=" * 60)

    collected_documents = []
    current_token_count = 0

    dataset_config = f"{CONFIG_DATE}.{lang}"
    streaming_dataset = load_dataset(
        'wikimedia/wikipedia',
        dataset_config,
        split='train',
        streaming=True
    )

    pbar = tqdm(desc=f"Tokens for {lang}", total=TARGET_TOKEN_COUNT, unit='tok')

    for doc in streaming_dataset:
        if current_token_count >= TARGET_TOKEN_COUNT:
            break

        text = doc.get('text', '')
        if not text:
            continue

        # ⚡ Optional: truncate very large docs for efficiency
        text = text[:5000]
        num_tokens = len(tokenizer.encode(text, add_special_tokens=False))

        if num_tokens == 0:
            continue

        collected_documents.append(text)
        current_token_count += num_tokens
        pbar.update(num_tokens)

    pbar.close()

    # --- Step 5: Save to file ---
    output_filepath = os.path.join(OUTPUT_DIR, f"wikipedia_1M_{lang}.txt")
    print(f"\n💾 Saving {len(collected_documents)} docs ({current_token_count:,} tokens) to {output_filepath}...")

    with open(output_filepath, 'w', encoding='utf-8') as f:
        f.write("\n\n".join(collected_documents))

    print(f"✅ Successfully saved file for language '{lang}'. ({current_token_count:,} tokens total)")

print("\n🎯 All languages processed successfully with 1M tokens each!")


Output directory for text files: data
Loading tokenizer for 'meta-llama/Llama-3.2-3B'...
✅ Tokenizer loaded successfully.

🌍 Processing language: en


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Tokens for en: 1000237tok [00:05, 176553.95tok/s]



💾 Saving 1002 docs (1,000,237 tokens) to data/wikipedia_1M_en.txt...
✅ Successfully saved file for language 'en'. (1,000,237 tokens total)

🌍 Processing language: hi


Tokens for hi: 1000854tok [00:04, 238108.15tok/s]                           


💾 Saving 564 docs (1,000,854 tokens) to data/wikipedia_1M_hi.txt...
✅ Successfully saved file for language 'hi'. (1,000,854 tokens total)

🎯 All languages processed successfully with 1M tokens each!


In [7]:
import torch
import os
from transformers import AutoTokenizer
from tqdm.auto import tqdm

# --- Configuration ---
PROJECT_PATH = ""
DATA_DIR = os.path.join(PROJECT_PATH, "data")
MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']

print("🔹 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✅ Tokenizer loaded successfully.")

# --- Tokenize each 1M-token dataset ---
for lang in LANGUAGES:
    input_filepath = os.path.join(DATA_DIR, f"wikipedia_1M_{lang}.txt")
    output_filepath = os.path.join(DATA_DIR, f"id.{lang}.1M.llama.pt")

    print("\n" + "=" * 70)
    print(f"🌍 Processing language: {lang}")
    print(f"📥 Reading from: {input_filepath}")
    print(f"💾 Saving to: {output_filepath}")
    print("=" * 70)

    if not os.path.exists(input_filepath):
        print(f"❌ ERROR: Input file not found at {input_filepath}")
        continue

    token_ids = []
    total_tokens = 0
    flush_interval = 500_000  # flush every 0.5M tokens

    with open(input_filepath, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc=f"Tokenizing {lang}", unit="line"):
            line = line.strip()
            if not line:
                continue

            # Encode text into token IDs
            ids = tokenizer.encode(line, add_special_tokens=False)
            if not ids:
                continue

            token_ids.extend(ids)
            total_tokens += len(ids)

            # Periodically flush to disk to prevent memory overflow
            if len(token_ids) >= flush_interval:
                torch.save(torch.LongTensor(token_ids), output_filepath)
                print(f"🌀 Flushed {len(token_ids):,} tokens to {output_filepath}")
                token_ids = []

    # --- Save remaining tokens ---
    if token_ids:
        torch.save(torch.LongTensor(token_ids), output_filepath)
        print(f"✅ Final flush: saved {len(token_ids):,} remaining tokens")

    print(f"✅ Token tensor for '{lang}' saved successfully.")
    print(f"📊 Total tokens processed: {total_tokens:,}")

print("\n🎯 All tokenization steps complete for 1M-token datasets!")


🔹 Loading tokenizer...
✅ Tokenizer loaded successfully.

🌍 Processing language: en
📥 Reading from: data/wikipedia_1M_en.txt
💾 Saving to: data/id.en.1M.llama.pt


Tokenizing en: 0line [00:00, ?line/s]

🌀 Flushed 500,048 tokens to data/id.en.1M.llama.pt
✅ Final flush: saved 491,293 remaining tokens
✅ Token tensor for 'en' saved successfully.
📊 Total tokens processed: 991,341

🌍 Processing language: hi
📥 Reading from: data/wikipedia_1M_hi.txt
💾 Saving to: data/id.hi.1M.llama.pt


Tokenizing hi: 0line [00:00, ?line/s]

🌀 Flushed 500,025 tokens to data/id.hi.1M.llama.pt
✅ Final flush: saved 494,521 remaining tokens
✅ Token tensor for 'hi' saved successfully.
📊 Total tokens processed: 994,546

🎯 All tokenization steps complete for 1M-token datasets!


In [10]:
import torch
import os

# --- Configuration ---
DATA_DIR = "data"  # folder containing your original 1M-token files
LANGUAGES = ["en", "hi"]
VALIDATION_RATIO = 0.05  # 5% of tokens for validation

# Loop over each language
for lang in LANGUAGES:
    # Original training file (1M tokens)
    train_path = os.path.join(DATA_DIR, f"id.{lang}.train.1M.llama.pt")

    if not os.path.exists(train_path):
        print(f"❌ File not found: {train_path}")
        continue

    # Load the full training token tensor
    all_tokens = torch.load(train_path)
    total_tokens = all_tokens.size(0)

    # Compute sizes
    val_size = int(total_tokens * VALIDATION_RATIO)
    train_size = total_tokens - val_size

    # Split tokens into train and validation
    train_tokens = all_tokens[:train_size]
    valid_tokens = all_tokens[train_size:]

    # Save new train and validation files
    train_out = os.path.join(DATA_DIR, f"id.{lang}.train.1M.llama.pt")
    valid_out = os.path.join(DATA_DIR, f"id.{lang}.valid.1M.llama.pt")

    torch.save(train_tokens, train_out)
    torch.save(valid_tokens, valid_out)

    print(f"✅ {lang.upper()} split:")
    print(f"   Train tokens: {train_size} → {train_out}")
    print(f"   Validation tokens: {val_size} → {valid_out}")


✅ EN split:
   Train tokens: 443393 → data/id.en.train.1M.llama.pt
   Validation tokens: 23336 → data/id.en.valid.1M.llama.pt
✅ HI split:
   Train tokens: 446306 → data/id.hi.train.1M.llama.pt
   Validation tokens: 23489 → data/id.hi.valid.1M.llama.pt


**activation.py**

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
import os

# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']
DATA_DIR = "data"
BATCH_SIZE = 2
MAX_LENGTH = 1024

print("🔹 Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()
print("✅ Model loaded successfully.")

# --- Extract model dimensions ---
num_layers = model.config.num_hidden_layers
intermediate_size = model.config.intermediate_size  # e.g. 8192
print(f"📊 Model has {num_layers} layers, intermediate size = {intermediate_size}")

# --- Initialize neuron activation counter ---
over_zero = torch.zeros(num_layers, intermediate_size, dtype=torch.int32, device='cuda')

# --- Hook for MLP activation tracking ---
def mlp_hook(layer_idx):
    def hook(module, input, output):
        # Compute full activation before down projection
        x = input[0]
        gate_up = module.gate_proj(x)
        up = module.up_proj(x)
        activation = (torch.nn.functional.silu(gate_up) * up).float()
        # Count how many activations > 0 per neuron
        over_zero[layer_idx, :activation.size(-1)] += (activation > 0).sum(dim=(0, 1))
    return hook

# Register hooks for all MLP layers
for i, layer in enumerate(model.model.layers):
    layer.mlp.register_forward_hook(mlp_hook(i))

# --- Process each language ---
for lang in LANGUAGES:
    tensor_path = os.path.join(DATA_DIR, f"id.{lang}.train.1M.llama.pt")
    if not os.path.exists(tensor_path):
        print(f"❌ ERROR: Token file not found at {tensor_path}")
        continue

    ids = torch.load(tensor_path).to('cuda')
    total_tokens = ids.size(0)
    print(f"\n🌍 Processing language: {lang} ({total_tokens:,} tokens)")

    # Process tokens in batches
    for start in tqdm(range(0, total_tokens, BATCH_SIZE * MAX_LENGTH), desc=f"🔄 Forward passes for {lang}"):
        batch_ids = []
        for b in range(BATCH_SIZE):
            s = start + b * MAX_LENGTH
            if s >= total_tokens:
                break
            e = min(s + MAX_LENGTH, total_tokens)
            chunk = ids[s:e]
            if chunk.size(0) < MAX_LENGTH:
                # Pad to full length
                pad = torch.zeros(MAX_LENGTH - chunk.size(0), dtype=chunk.dtype, device='cuda')
                chunk = torch.cat([chunk, pad])
            batch_ids.append(chunk)

        if not batch_ids:
            continue

        batch_ids = torch.stack(batch_ids, dim=0)
        attention_mask = (batch_ids != 0).long()

        with torch.no_grad():
            _ = model(input_ids=batch_ids, attention_mask=attention_mask)

    # --- Save activation statistics ---
    output_path = os.path.join(DATA_DIR, f"activation.{lang}.train.1M.llama3b.pt")
    torch.save({'over_zero': over_zero.cpu()}, output_path)
    print(f"✅ Saved activations for '{lang}' → {output_path}")

print("\n🎯 Activation collection completed for all languages (1M-token datasets).")


🔹 Loading model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully.
📊 Model has 28 layers, intermediate size = 8192

🌍 Processing language: en (443,393 tokens)


🔄 Forward passes for en:   0%|          | 0/217 [00:00<?, ?it/s]

✅ Saved activations for 'en' → data/activation.en.train.1M.llama3b.pt

🌍 Processing language: hi (446,306 tokens)


🔄 Forward passes for hi:   0%|          | 0/218 [00:00<?, ?it/s]

✅ Saved activations for 'hi' → data/activation.hi.train.1M.llama3b.pt

🎯 Activation collection completed for all languages (1M-token datasets).


In [6]:
import torch
import numpy as np
import pandas as pd
import os

# --- File path ---
file_path = 'data/activation.hi.train.1M.llama3b.pt'

# --- Load the .pt file ---
print(f"Loading tensor from: {file_path}")
data = torch.load(file_path, map_location='cpu')

# --- Extract the 'over_zero' tensor ---
if 'over_zero' not in data:
    raise KeyError("❌ Key 'over_zero' not found in the loaded file. Available keys: " + str(list(data.keys())))

over_zero = data['over_zero'].cpu()  # Expected shape: 28 x 8192
print(f"✅ Loaded 'over_zero' tensor with shape: {tuple(over_zero.shape)}")

# --- Convert to NumPy for easier handling ---
matrix = over_zero.numpy()

# --- Show summary to avoid console flooding ---
print("\n🔹 Tensor Summary:")
print(f"Shape: {matrix.shape}")
print(f"Min: {matrix.min()}, Max: {matrix.max()}, Mean: {matrix.mean()}")
print("\n🔹 Sample (first 3 rows):")
print(matrix[:3])  # print only first 3 rows for readability

# --- Save to CSV ---
csv_path = file_path.replace(".pt", ".csv")
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df = pd.DataFrame(matrix)
df.to_csv(csv_path, index=False)

print(f"\n✅ over_zero tensor successfully saved to CSV:")
print(f"📂 {csv_path}")


Loading tensor from: data/activation.hi.train.1M.llama3b.pt
✅ Loaded 'over_zero' tensor with shape: (28, 8192)

🔹 Tensor Summary:
Shape: (28, 8192)
Min: 0, Max: 890880, Mean: 445263.85991123744

🔹 Sample (first 3 rows):
[[435993 436378 434422 ... 329206 424425 438842]
 [457913 449732 440073 ... 308689 436481 321708]
 [477596 431045 451697 ... 448848 457595 498834]]

✅ over_zero tensor successfully saved to CSV:
📂 data/activation.hi.train.1M.llama3b.csv


In [7]:

# --- File path ---
file_path = 'data/activation.en.train.1M.llama3b.pt'

# --- Load the .pt file ---
print(f"Loading tensor from: {file_path}")
data = torch.load(file_path, map_location='cpu')

# --- Extract the 'over_zero' tensor ---
if 'over_zero' not in data:
    raise KeyError("❌ Key 'over_zero' not found in the loaded file. Available keys: " + str(list(data.keys())))

over_zero = data['over_zero'].cpu()  # Expected shape: 28 x 8192
print(f"✅ Loaded 'over_zero' tensor with shape: {tuple(over_zero.shape)}")

# --- Convert to NumPy for easier handling ---
matrix = over_zero.numpy()

# --- Show summary to avoid console flooding ---
print("\n🔹 Tensor Summary:")
print(f"Shape: {matrix.shape}")
print(f"Min: {matrix.min()}, Max: {matrix.max()}, Mean: {matrix.mean()}")
print("\n🔹 Sample (first 3 rows):")
print(matrix[:3])  # print only first 3 rows for readability

# --- Save to CSV ---
csv_path = file_path.replace(".pt", ".csv")
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df = pd.DataFrame(matrix)
df.to_csv(csv_path, index=False)

print(f"\n✅ over_zero tensor successfully saved to CSV:")
print(f"📂 {csv_path}")


Loading tensor from: data/activation.en.train.1M.llama3b.pt
✅ Loaded 'over_zero' tensor with shape: (28, 8192)

🔹 Tensor Summary:
Shape: (28, 8192)
Min: 0, Max: 444416, Mean: 222106.46555873327

🔹 Sample (first 3 rows):
[[213640 237007 202883 ... 164857 214296 238791]
 [236456 215944 212300 ... 131038 216587 139884]
 [231557 228562 232253 ... 220941 218247 229130]]

✅ over_zero tensor successfully saved to CSV:
📂 data/activation.en.train.1M.llama3b.csv


**identify.py**

In [8]:
import torch
import os

# --- Configuration ---
languages = ['en', 'hi']  # your hi-en dataset
activation_dir = 'data'
output_dir = 'activation_mask'
os.makedirs(output_dir, exist_ok=True)

top_rate = 0.01           # fraction of neurons to select (entropy-based)
filter_rate = 0.95        # top 5% neurons threshold
activation_bar_ratio = 0.95  # top 5% activation threshold

# --- Load activation data ---
over_zero = []
for lang in languages:
    file_path = os.path.join(activation_dir, f'activation.{lang}.train.1M.llama3b.pt')
    data = torch.load(file_path)
    over_zero.append(data['over_zero'])

# shape: [layers, neurons, languages]
over_zero = torch.stack(over_zero, dim=-1)
num_layers, intermediate_size, lang_num = over_zero.size()
print(f"{lang_num} languages, {num_layers} layers, {intermediate_size} neurons per layer")

# --- Compute activation probabilities across languages ---
activation_probs = over_zero / over_zero.sum(dim=-1, keepdim=True)
activation_probs[torch.isnan(activation_probs)] = 0

# --- Compute entropy across languages ---
log_probs = torch.where(activation_probs > 0, activation_probs.log(), 0)
entropy = -torch.sum(activation_probs * log_probs, dim=-1)  # shape: [layers, neurons]

# --- Filter neurons with very low activation ---
flattened_probs = activation_probs.flatten()
top_prob_value = flattened_probs.kthvalue(round(len(flattened_probs) * filter_rate)).values.item()
active_neurons = (activation_probs > top_prob_value).sum(dim=-1)
entropy[active_neurons == 0] = torch.inf  # ignore inactive neurons

# --- Select top neurons by smallest entropy (language-specific) ---
flattened_entropy = entropy.flatten()
top_entropy_count = round(len(flattened_entropy) * top_rate)
entropy_values, index = flattened_entropy.topk(top_entropy_count, largest=False)  # smallest entropy = language-specific

row_index = index // entropy.size(1)
col_index = index % entropy.size(1)

# --- Determine which language dominates each top neuron ---
# For each selected neuron (layer,row_index,col_index),
# check which language has the higher activation probability
dominant_lang = []
for r, c in zip(row_index.tolist(), col_index.tolist()):
    probs = activation_probs[r, c]  # shape [languages]
    winner = probs.argmax().item()  # 0 -> en, 1 -> hi
    dominant_lang.append(winner)

dominant_lang = torch.tensor(dominant_lang)
counts = torch.bincount(dominant_lang, minlength=len(languages))

total = counts.sum().item()
print("\n🔍 Top 1% language dominance based on entropy:")
for i, lang in enumerate(languages):
    pct = 100 * counts[i].item() / total if total > 0 else 0
    print(f"   {lang.upper():>3}: {counts[i].item()} neurons ({pct:.2f}%)")

# --- Build and save per-language masks separately (same as before) ---
for lang_id, lang_name in enumerate(languages):
    lang_probs = activation_probs[:, :, lang_id]
    selected_mask = torch.zeros_like(lang_probs, dtype=torch.bool)
    selected_mask[row_index, col_index] = True
    activation_bar = flattened_probs.kthvalue(round(len(flattened_probs) * activation_bar_ratio)).values.item()
    selected_mask &= (lang_probs > activation_bar)

    layer_indices = []
    for layer in range(num_layers):
        neurons = torch.where(selected_mask[layer])[0]
        layer_indices.append(neurons)

    output_file = os.path.join(output_dir, f"llama3b_{lang_name}_mask.pt")
    torch.save(layer_indices, output_file)
    print(f"✅ Activation mask for {lang_name} saved to: {output_file}")


2 languages, 28 layers, 8192 neurons per layer

🔍 Top 1% language dominance based on entropy:
    EN: 0 neurons (0.00%)
    HI: 2294 neurons (100.00%)
✅ Activation mask for en saved to: activation_mask/llama3b_en_mask.pt
✅ Activation mask for hi saved to: activation_mask/llama3b_hi_mask.pt


In [9]:
import torch
import os

# --- Configuration ---
output_dir = 'activation_mask'
languages = ['en', 'hi']

for lang in languages:
    file_path = os.path.join(output_dir, f'llama3b_{lang}_mask.pt')

    # Load the saved mask
    layer_masks = torch.load(file_path)

    print(f"\n--- Activation mask for {lang.upper()} ---")
    print("layer-wise neuron indices = [")

    for i, neuron_tensor in enumerate(layer_masks):
        print(f"    layer {i}: tensor({neuron_tensor.tolist()})")

    print("]\n")



--- Activation mask for EN ---
layer-wise neuron indices = [
    layer 0: tensor([])
    layer 1: tensor([])
    layer 2: tensor([])
    layer 3: tensor([])
    layer 4: tensor([])
    layer 5: tensor([])
    layer 6: tensor([])
    layer 7: tensor([])
    layer 8: tensor([])
    layer 9: tensor([])
    layer 10: tensor([])
    layer 11: tensor([])
    layer 12: tensor([])
    layer 13: tensor([])
    layer 14: tensor([])
    layer 15: tensor([])
    layer 16: tensor([])
    layer 17: tensor([])
    layer 18: tensor([])
    layer 19: tensor([])
    layer 20: tensor([])
    layer 21: tensor([])
    layer 22: tensor([])
    layer 23: tensor([])
    layer 24: tensor([])
    layer 25: tensor([])
    layer 26: tensor([])
    layer 27: tensor([])
]


--- Activation mask for HI ---
layer-wise neuron indices = [
    layer 0: tensor([268, 513, 988, 1365, 1652, 1934, 2353, 2788, 2815, 2990, 3451, 3936, 4278, 5281, 5758, 5943, 6197, 6237, 6248, 6384, 7374, 7503, 8004])
    layer 1: tensor([46, 4

**ppl.py**

In [11]:
import argparse
from types import MethodType
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# -------------------- Argument Parser --------------------
parser = argparse.ArgumentParser(description="Evaluate LLaMA model with language-specific activation masks.")
parser.add_argument(
    "-m", "--model",
    type=str,
    default="meta-llama/Llama-3.2-3B",
    help="Model name or path (default: meta-llama/Llama-3.2-3B)"
)
parser.add_argument(
    "-d", "--data_dir",
    type=str,
    default="data",
    help="Directory containing tokenized validation data (id.<lang>.valid.llama.pt)"
)
parser.add_argument(
    "-a", "--activation_mask_dir",
    type=str,
    default="activation_mask",
    help="Directory containing per-language activation masks (llama3b_<lang>_mask.pt)"
)
args, unknown = parser.parse_known_args()
if unknown:
    print(f"Ignoring unknown arguments: {unknown}")

# -------------------- Load Model & Tokenizer --------------------
print("🔹 Loading model...")
tokenizer = AutoTokenizer.from_pretrained(args.model)
model = AutoModelForCausalLM.from_pretrained(
    args.model,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()
print("✅ Model loaded.")

num_layers = model.config.num_hidden_layers
intermediate_size = model.config.intermediate_size
max_length = model.config.max_position_embeddings

print(f"📊 Layers: {num_layers}, Hidden neurons per MLP: {intermediate_size}")

# -------------------- Languages --------------------
languages = ["en", "hi"]

# Load activation masks
activation_masks = []
for lang in languages:
    mask_path = os.path.join(args.activation_mask_dir, f"llama3b_{lang}_mask.pt")
    if os.path.exists(mask_path):
        print(f"🔹 Found activation mask for {lang}: {mask_path}")
        activation_masks.append(torch.load(mask_path))
    else:
        print(f"⚠️ No activation mask found for {lang}, using None")
        activation_masks.append(None)

# -------------------- Helper: Forward Patch Factory --------------------
def factory(mask):
    """Custom forward method that zeroes out masked neurons."""
    def llama_forward(self, x):
        gate_up = self.gate_proj(x)
        up = self.up_proj(x)
        activation = F.silu(gate_up) * up

        # Zero out masked neurons
        if mask is not None and mask.numel() > 0:
            activation.index_fill_(2, mask.to(x.device), 0)

        return self.down_proj(activation)
    return llama_forward

# -------------------- Evaluation --------------------
final_output = []

for activation_mask, mask_lang in zip(activation_masks, languages):
    print(f"\n🔧 Applying activation mask for: {mask_lang.upper()}")

    # Apply mask to each MLP layer
    if activation_mask is not None:
        for i, layer_mask in enumerate(activation_mask):
            if layer_mask.numel() == 0:
                continue
            obj = model.model.layers[i].mlp
            obj.forward = MethodType(factory(layer_mask), obj)

    # -------------------- Evaluate PPL across languages --------------------
    ppls = []
    for lang in languages:
        val_path = os.path.join(args.data_dir, f"id.{lang}.valid.1M.llama.pt")
        if not os.path.exists(val_path):
            print(f"❌ Missing validation file: {val_path}")
            ppls.append(float("nan"))
            continue

        ids = torch.load(val_path)
        total_len = ids.size(0)
        max_len = min(max_length, 1024)
        total_len = (total_len // max_len) * max_len
        input_ids = ids[:total_len].reshape(-1, max_len)

        print(f"   🔹 Evaluating {lang.upper()} ({input_ids.shape[0]} sequences)...")

        losses = []
        for batch in input_ids.to(model.device):
            with torch.no_grad():
                out = model(batch.unsqueeze(0), labels=batch.unsqueeze(0))
                losses.append(out.loss.item())

        mean_loss = np.mean(losses)
        ppls.append(mean_loss)

    final_output.append(ppls)

# -------------------- Print Results --------------------
print("\n📊 Average Negative Log-Likelihood (proxy for PPL):")
header = "MASK_LANG | " + " | ".join([f"{l.upper():>5}" for l in languages])
print(header)
print("-" * len(header))

for mask_lang, ppls in zip(languages, final_output):
    row = f"{mask_lang.upper():>9} | " + " | ".join([f"{ppl:.3f}" if not np.isnan(ppl) else "  N/A " for ppl in ppls])
    print(row)


Ignoring unknown arguments: ['-f', '/root/.local/share/jupyter/runtime/kernel-bd42ecb1-6f72-46bc-bc20-af423b9f2ca3.json']
🔹 Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded.
📊 Layers: 28, Hidden neurons per MLP: 8192
🔹 Found activation mask for en: activation_mask/llama3b_en_mask.pt
🔹 Found activation mask for hi: activation_mask/llama3b_hi_mask.pt

🔧 Applying activation mask for: EN
   🔹 Evaluating EN (22 sequences)...
   🔹 Evaluating HI (22 sequences)...

🔧 Applying activation mask for: HI
   🔹 Evaluating EN (22 sequences)...
   🔹 Evaluating HI (22 sequences)...

📊 Average Negative Log-Likelihood (proxy for PPL):
MASK_LANG |    EN |    HI
-------------------------
       EN | 2.162 | 1.579
       HI | 2.317 | 1.715


**Generate.py**

In [14]:
# List of prompts
prompts = [
    "How can I improve my time management skills?",
    "What are the most effective ways to deal with stress?",
    "What are the main differences between Python and JavaScript programming languages?",
    "How can I increase my productivity while working from home?",
    "Can you explain the basics of quantum computing?",
    "What are the differences between plant-based and animal-based protein sources?",
    "How can I develop my critical thinking skills?",
    "What are the major challenges faced by the education sector today?",
    "What are the primary factors that influence consumer behavior?",
    "What are the most effective strategies for conflict resolution in the workplace?",
    "What are some potential implications of using a single-use plastic bottle versus a reusable bottle on both the environment and human health?",
    "What factors would you consider when designing an inclusive and accessible public transportation system?",
    "How can governments utilize fiscal and monetary policies to combat economic recessions?",
    "How do language and cultural barriers affect the way people communicate and form relationships in multicultural societies?",
    "Describe a scenario where artificial intelligence could be used to improve the quality and efficiency of healthcare delivery.",
    "Explain the process of gene editing using CRISPR-Cas9 technology, and discuss its potential applications and ethical implications.",
    "How do vaccinations work to protect individuals and communities from infectious diseases, and what is herd immunity?",
    "How do social media platforms influence the way people consume and share news, and what are the potential implications for the spread of misinformation?",
    "How do cultural, social, and economic factors influence people's food choices, and how can this knowledge be used to promote healthier diets?",
    "Explain the process of natural selection and how it contributes to the evolution and adaptation of species.",
    "How would you introduce yourself as a medieval knight at a royal banquet?",
    "As a pirate captain, what would you say to your crew to motivate them to search for hidden treasure?",
    "If you were a Shakespearean character, how would you declare your love for someone in a soliloquy?",
    "As a superhero, how would you explain your origin story to a curious child?",
    "Imagine you are a time traveler from the year 3000. What technological advancements would you tell people about?",
    "As a sports commentator, describe the winning play in the final seconds of a championship game.",
    "Pretend to be a world-famous chef. How would you describe your signature dish to a panel of judges?",
    "You are a mountain climber reaching the summit of Mount Everest. Describe your emotions and the view from the top.",
    "As a space colonist on Mars, describe your daily life and the challenges you face living on another planet.",
    "Pretend to be a character in a post-apocalyptic world. Describe how you survive and the allies you encounter.",
    "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?",
    "What are some subtle clues that suggest someone is pretending to understand a topic or conversation when they are actually confused or uninformed?",
    "Why might someone choose to use a paper map or ask for directions instead of relying on a GPS device or smartphone app?",
    "How can you determine if a person is genuinely interested in a conversation or simply being polite?",
    "Why might someone prefer to shop at a small, locally-owned business instead of a large chain store, even if the prices are higher?",
    "How can you assess the credibility of a source of information, such as a news article or blog post, without relying solely on the reputation of the author or publisher?",
    "Why do some people enjoy the sensation of being scared, such as by watching horror movies or going on roller coasters, while others avoid these experiences?",
    "How can observing the behavior of other people in a social situation provide clues about cultural norms and expectations?",
    "Do we have a moral obligation to explore space, or should we focus on solving Earth's problems first?",
    "In a world where automation is becoming increasingly prevalent, is it more important to prioritize job creation or technological progress?",
    "How many times does the average human blink in a lifetime? Try to explain your answer step-by-step.",
    "How many atoms are in a grain of salt? Try to explain your answer step-by-step.",
    "How many lightning strikes occur on Earth each day? Try to explain your answer step-by-step.",
    "How many balloons would it take to lift a house like in the movie 'Up'? Try to explain your answer step-by-step.",
    "How many text messages are sent globally in a minute? Try to explain your answer step-by-step.",
    "How many words are spoken daily on Earth? Try to explain your answer step-by-step.",
    "How many snowflakes fall during a typical winter? Try to explain your answer step-by-step.",
    "How many pages are in all the books ever written? Try to explain your answer step-by-step.",
    "How many times has the Earth orbited the Sun since the beginning of life? Try to explain your answer step-by-step.",
    "How many songs have been recorded throughout history? Try to explain your answer step-by-step.",
    "What if the Internet had been invented during the Renaissance period?",
    "What if the Aztecs had successfully repelled the Spanish conquistadors?",
    "What if the Black Death had not occurred in the 14th century?",
    "What if Isaac Newton had focused on biology instead of physics?",
    "What if the Beatles had never formed as a band?",
    "What if Alan Turing had not cracked the Enigma code during World War II?",
    "What if the Suez Canal had never been constructed?",
    "What if the Maya civilization had never mysteriously collapsed?",
    "What if Christopher Columbus had not discovered the Americas?",
    "What if Vincent van Gogh had been a successful artist during his lifetime?",
    "Can you help me write a formal email to a potential business partner proposing a joint venture?",
    "Can you help me write a resignation letter to my current employer, while leaving on good terms and expressing gratitude for the opportunities provided?",
    "Use an appropriate format to structure a formal letter of recommendation for a student applying to a prestigious graduate program in computer science.",
    "Write a compelling product launch announcement email to inform our customers of our new software solution.",
    "Draft an apology email to a customer who experienced a delay in their order, and provide reassurance that the issue has been resolved.",
    "Write a script for a YouTube video exploring the history and cultural significance of jazz.",
    "Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.",
    "Write a captivating movie review for a recently released science fiction film, discussing its plot, characters, and special effects.",
    "Structure a podcast script for an episode discussing the influence of streaming platforms on the music industry.",
    "Write a symphony concert review, discussing the orchestra's performance and overall audience experience."
]

# Create directory if it doesn't exist
os.makedirs("dataset/mvicuna", exist_ok=True)

# Write prompts to en.txt
with open("dataset/mvicuna/en.txt", "w", encoding="utf-8") as f:
    for prompt in prompts:
        f.write(prompt + "\n")

print("✅ Saved prompts to dataset/mvicuna/en.txt")


✅ Saved prompts to dataset/mvicuna/en.txt


In [15]:
hi_prompts = [
    # Self-improvement
    "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ?",
    "तनाव से निपटने के सबसे प्रभावी तरीके क्या हैं?",
    "मैं अपनी आलोचनात्मक सोच कौशल कैसे विकसित कर सकता हूँ?",
    "घर से काम करते समय अपनी उत्पादकता कैसे बढ़ा सकता हूँ?",
    "सकारात्मक आदतें विकसित करने के लिए क्या कदम उठाए जा सकते हैं?",

    # Science & Technology
    "क्वांटम कंप्यूटिंग की मूल बातें समझाइए।",
    "CRISPR-Cas9 तकनीक के माध्यम से जीन संपादन की प्रक्रिया बताइए।",
    "वैक्सीन कैसे काम करती हैं और हर्ड इम्युनिटी क्या है?",
    "प्लास्टिक की एकल-उपयोग वाली बोतल और पुन: प्रयोज्य बोतल का पर्यावरण और स्वास्थ्य पर क्या प्रभाव पड़ता है?",
    "Python और JavaScript प्रोग्रामिंग भाषाओं के मुख्य अंतर क्या हैं?",

    # Daily life & practical scenarios
    "घर से काम करते समय समय और ऊर्जा का प्रबंधन कैसे करें?",
    "सामाजिक स्थितियों में दूसरों के व्यवहार को देखकर सांस्कृतिक नियम कैसे समझे जा सकते हैं?",
    "एक रेस्टोरेंट की लोकप्रियता को कैसे परखा जा सकता है?",
    "किसी स्रोत की विश्वसनीयता का मूल्यांकन कैसे किया जाए?",
    "व्यक्तिगत और व्यावसायिक जीवन में ध्यान और फोकस कैसे बनाए रखें?",

    # Imaginative & creative scenarios
    "एक समुद्री डाकू कप्तान के रूप में अपने क्रू को खजाना खोजने के लिए कैसे प्रोत्साहित करेंगे?",
    "यदि आप एक सुपरहीरो हों, तो अपने उत्पत्ति की कहानी कैसे बताएंगे?",
    "कल्पना कीजिए कि आप वर्ष 3000 से समय यात्री हैं। लोगों को किन तकनीकी विकासों के बारे में बताएंगे?",
    "एक अंतरिक्ष उपनिवेशकर्ता के रूप में मंगल पर अपने दैनिक जीवन और चुनौतियों का वर्णन कीजिए।",
    "पोस्ट-एपोकैलिप्टिक दुनिया में एक पात्र के रूप में अपने उत्तरजीविता कौशल का वर्णन करें।",

    # History & what-if scenarios
    "यदि ब्लैक डेथ 14वीं सदी में नहीं हुआ होता तो क्या होता?",
    "अगर बीटल्स कभी एक बैंड के रूप में नहीं बने होते तो संगीत उद्योग पर क्या प्रभाव पड़ता?",
    "यदि क्रिस्टोफर कोलंबस ने अमेरिका की खोज नहीं की होती तो इतिहास कैसे बदलता?",
    "यदि माया सभ्यता अचानक ढह न जाती तो क्या होता?",
    "यदि न्यूटन ने भौतिकी के बजाय जीवविज्ञान पर ध्यान केंद्रित किया होता तो क्या होता?",

    # Science & estimation questions
    "एक मानक वर्ष में पृथ्वी पर कितने बिजली गिरते हैं?",
    "एक साल में मानव शरीर में औसतन कितनी बार पलक झपकते हैं?",
    "एक ग्रेन नमक में कितने परमाणु होते हैं? इसका अनुमान बताइए।",
    "एक मिनट में दुनिया भर में कितने टेक्स्ट संदेश भेजे जाते हैं?",
    "पृथ्वी पर प्रतिदिन कितने शब्द बोले जाते हैं? इसका अनुमान लगाइए।",

    # Practical writing tasks
    "एक व्यवसायिक सहयोग के लिए औपचारिक ईमेल कैसे लिखा जाए?",
    "ग्राहक को देर हुए आदेश के लिए माफी का पत्र कैसे लिखा जाए?",
    "कंप्यूटर विज्ञान में स्नातक छात्र के लिए सिफारिश पत्र कैसे लिखा जाए?",
    "नई सॉफ़्टवेयर सुविधा की घोषणा के लिए आकर्षक ईमेल कैसे लिखा जाए?",
    "यूट्यूब वीडियो के लिए स्क्रिप्ट कैसे तैयार की जाए जो इतिहास और सांस्कृतिक महत्व पर हो?",

    # Food & lifestyle
    "सांस्कृतिक, सामाजिक और आर्थिक कारक लोगों की भोजन पसंद को कैसे प्रभावित करते हैं?",
    "शाकाहारी और मांसाहारी प्रोटीन स्रोतों के बीच मुख्य अंतर क्या हैं?",
    "स्वस्थ आहार को बढ़ावा देने के लिए लोगों के भोजन चुनाव का विश्लेषण कैसे किया जाए?",
    "आप किसी रेस्तरां में अपने अनुभव और व्यंजन का वर्णन कैसे करेंगे?",
    "स्थानीय बाजार में खरीदारी करने के क्या फायदे हैं, और लोग इसे क्यों चुनते हैं?",

    # Miscellaneous fun & curiosity
    "यदि आप मध्यकालीन शूरवीर होते, तो शाही भोज में खुद को कैसे पेश करते?",
    "यदि आप शेक्सपियरियन पात्र होते, तो किसी से प्रेम कैसे घोषित करते?",
    "यदि आप प्रसिद्ध शेफ होते, तो अपनी खास डिश का वर्णन कैसे करते?",
    "यदि आप समय यात्री होते, तो प्राचीन सभ्यताओं को क्या ज्ञान देते?",
    "यदि आप स्पेस कमांडर होते, तो मंगल पर जीवन का दैनिक विवरण कैसे देंगे?"
]

import os
os.makedirs("dataset/mvicuna", exist_ok=True)
with open("dataset/mvicuna/hi.txt", "w", encoding="utf-8") as f:
    for prompt in hi_prompts:
        f.write(prompt + "\n")

print("✅ Hindi prompts saved to dataset/mvicuna/hi.txt")


✅ Hindi prompts saved to dataset/mvicuna/hi.txt


In [1]:
import json
import os
from types import MethodType

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# -------------------- Configuration --------------------
MODEL_NAME = "meta-llama/Llama-3.2-3B"
DATA_DIR = "dataset/mvicuna"  # folder with your question datasets per language
ACTIVATION_MASK_DIR = "activation_mask"  # folder with per-language masks
OUTPUT_DIR = f"results/{MODEL_NAME.split('/')[-1]}/mvicuna"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LANGUAGES = ["en", "hi"]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

answer_lang = {
    "en": " Answer in English.",
    "hi": " Answer in Hindi.",  # you can customize or use the same as English
}

# -------------------- Load model and tokenizer --------------------
print("🔹 Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()
print("✅ Model loaded.")
num_layers = model.config.num_hidden_layers
intermediate_size = model.config.intermediate_size
print(f"📊 Layers: {num_layers}, Hidden neurons per MLP: {intermediate_size}")

# -------------------- Load activation masks --------------------
activation_masks = {}
for lang in LANGUAGES:
    mask_path = os.path.join(ACTIVATION_MASK_DIR, f"llama3b_{lang}_mask.pt")
    if os.path.exists(mask_path):
        activation_masks[lang] = torch.load(mask_path)
        print(f"🔹 Found activation mask for {lang}: {mask_path}")
    else:
        activation_masks[lang] = None
        print(f"⚠️ No activation mask found for {lang}")

# -------------------- Helper: Patch forward function --------------------
def mlp_forward_factory(mask):
    """Return a patched forward method that zeros out neurons in the mask."""
    def forward(self, x):
        gate_up = self.gate_proj(x)
        up = self.up_proj(x)
        activation = F.silu(gate_up) * up
        if mask is not None and mask.numel() > 0:
            activation.index_fill_(2, mask.to(x.device), 0)
        return self.down_proj(activation)
    return forward

# -------------------- Run inference per mask --------------------
for mask_lang, mask in activation_masks.items():
    # Patch model MLPs for this mask
    if mask is not None:
        for i, layer_mask in enumerate(mask):
            if layer_mask.numel() == 0:
                continue
            mlp_layer = model.model.layers[i].mlp
            mlp_layer.forward = MethodType(mlp_forward_factory(layer_mask), mlp_layer)

    # Run inference on EN and HI datasets
    for lang in LANGUAGES:
        input_file = os.path.join(DATA_DIR, f"{lang}.txt")
        if not os.path.exists(input_file):
            print(f"❌ Missing dataset for {lang}: {input_file}")
            continue

        with open(input_file, "r", encoding="utf-8") as f:
            texts = [line.strip() + answer_lang[lang] for line in f if line.strip()]
            texts = [f"Q: {t}\nA:" for t in texts]

        results = []
        for t in tqdm(texts, desc=f"{lang} <- mask {mask_lang}"):
            input_ids = tokenizer(t, return_tensors="pt").input_ids.to(DEVICE)
            with torch.no_grad():
                output_ids = model.generate(input_ids, max_new_tokens=256)
            output_text = tokenizer.decode(output_ids[0][input_ids.size(1):], skip_special_tokens=True)
            results.append({"input": t, "output": output_text})

        # Save results
        if mask is not None:
            out_file = os.path.join(OUTPUT_DIR, f"{lang}.perturb.{mask_lang}.jsonl")
        else:
            out_file = os.path.join(OUTPUT_DIR, f"{lang}.jsonl")

        with open(out_file, "w", encoding="utf-8") as f:
            for r in results:
                f.write(json.dumps(r, ensure_ascii=False) + "\n")

        print(f"✅ Saved results: {out_file}")


🔹 Loading model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded.
📊 Layers: 28, Hidden neurons per MLP: 8192
🔹 Found activation mask for en: activation_mask/llama3b_en_mask.pt
🔹 Found activation mask for hi: activation_mask/llama3b_hi_mask.pt


en <- mask en:   0%|          | 0/70 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
en <- mask en:   1%|▏         | 1/70 [00:12<13:54, 12.09s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
en <- mask en:   3%|▎         | 2/70 [00:22<12:36, 11.12s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unex

✅ Saved results: results/Llama-3.2-3B/mvicuna/en.perturb.en.jsonl


hi <- mask en:   0%|          | 0/45 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
hi <- mask en:   2%|▏         | 1/45 [00:10<07:35, 10.35s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
hi <- mask en:   4%|▍         | 2/45 [00:20<07:26, 10.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
hi <- mask en:   7%|▋         | 3/45 [00:31<07:17, 10.42s/it]The atte

KeyboardInterrupt: 

In [7]:
import shutil
from google.colab import files

# Replace 'folder_name' with your folder
folders_to_download = ['data', 'dataset']

for folder in folders_to_download:
    zip_name = f"{folder}.zip"
    shutil.make_archive(folder, 'zip', folder)
    files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from google.colab import drive
import shutil

# Mount your Google Drive
drive.mount('/content/drive')

folders_to_download = ['data', 'dataset']

for folder in folders_to_download:
    zip_name = f"/content/drive/MyDrive/{folder}.zip"
    shutil.make_archive(f"/content/{folder}", 'zip', folder)
    shutil.move(f"/content/{folder}.zip", zip_name)
    print(f"✅ {folder} saved to Google Drive: {zip_name}")


Mounted at /content/drive
✅ data saved to Google Drive: /content/drive/MyDrive/data.zip
✅ dataset saved to Google Drive: /content/drive/MyDrive/dataset.zip
